#### Question 1: Embedding the query

In [1]:
from fastembed import TextEmbedding
import numpy as np

In [2]:
model = "jinaai/jina-embeddings-v2-small-en"

In [3]:
q1_query = 'I just discovered the course. Can I join now?'

In [4]:
# https://qdrant.github.io/fastembed/Getting%20Started/#quick-start
embedding_model = TextEmbedding(model_name=model)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

In [5]:
q1_query_embedding = next(embedding_model.embed(q1_query))

In [6]:
# embeddings_generator = model.embed(query)

# for doc, vector in zip(query, embeddings_generator):
#     print("Document:", doc)
#     print(f"Vector of type: {type(vector)} with shape: {vector.shape}")

In [7]:
q1_embedding_np_array = np.array(q1_query_embedding)

In [8]:
print("Shape:", q1_embedding_np_array.shape) 

Shape: (512,)


In [9]:
print("Min value:", q1_embedding_np_array.min())

Min value: -0.11726373885183883


#### Question 2: Cosine similarity

In [10]:
np.linalg.norm(q1_embedding_np_array)

np.float64(1.0)

In [11]:
q1_embedding_np_array.dot(q1_embedding_np_array)

np.float64(1.0000000000000002)

In [12]:
q2_query = 'Can I still join the course after the start date?'

In [13]:
q2_query_embedding = next(embedding_model.embed(q2_query))

In [14]:
q2_embedding_np_array = np.array(q2_query_embedding)

In [15]:
q1_embedding_np_array.dot(q2_embedding_np_array)

np.float64(0.9008528895674548)

In [16]:
q2_embedding_np_array.dot(q1_embedding_np_array)

np.float64(0.9008528895674548)

In [17]:
print("Cosine similarity:", q2_embedding_np_array.dot(q1_embedding_np_array))

Cosine similarity: 0.9008528895674548


#### Question 3: Ranking by Cosine

In [18]:
q3_documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [19]:
import numpy as np

texts = [doc["text"] for doc in q3_documents]
vectors = list(embedding_model.embed(texts))

# Combine as pairs: [text, vector]
text_embeddings = np.array([
    [text, vector] for text, vector in zip(texts, vectors)
], dtype=object)
print(text_embeddings.shape)

(5, 2)


In [34]:
import pandas as pd
df_q3_text_embeddings = pd.DataFrame(text_embeddings,columns=['text','vector'])
# df_q3_text_embeddings

In [22]:
q3_query = "I just discovered the course. Can I join now?"
q3_query_vector = next(embedding_model.embed([q3_query]))  # Shape: (512,)
q3_query_vector.shape

(512,)

In [35]:
# Compute cosine similarity via dot product (vectors must be normalized)
df_q3_text_embeddings["similarity"] = df_q3_text_embeddings["vector"].apply(lambda x: np.dot(x, q3_query_vector))

# Sort to get the most similar at the top
df_q3_sorted = df_q3_text_embeddings.sort_values(by="similarity", ascending=False)

df_q3_sorted

,text,vector,similarity
1,"Yes, we will keep all the materials after the ...","[-0.05947089729865029, -0.08523984248531108, 0...",0.818238
2,The purpose of this document is to capture fre...,"[-0.06846454499342787, -0.0407909771755291, 0....",0.808540
0,"Yes, even if you don't register, you're still ...","[-0.024952500601280857, -0.039645403430715875,...",0.762968
4,Star the repo! Share it with friends if you fi...,"[-0.053949116022754706, -0.04693814362572923, ...",0.730450
3,You can start by installing and setting up all...,"[-0.0464050626316478, -0.025552290220715833, 0...",0.713308


#### Question 4 : Ranking by cosine, version two

In [36]:
import numpy as np

q4_texts = [doc['question'] + ' ' + doc['text'] for doc in q3_documents]
q4_vectors = list(embedding_model.embed(q4_texts))

# Combine as pairs: [text, vector]
q4_text_embeddings = np.array([
    [text, vector] for text, vector in zip(q4_texts, q4_vectors)
], dtype=object)
print(q4_text_embeddings.shape)
df_q4_text_embeddings = pd.DataFrame(q4_text_embeddings,columns=['text','vector'])
# df_q4_text_embeddings

(5, 2)


In [37]:
# Compute cosine similarity via dot product (vectors must be normalized)
df_q4_text_embeddings["similarity"] = df_q4_text_embeddings["vector"].apply(lambda x: np.dot(x, q3_query_vector))

# Sort to get the most similar at the top
df_q4_sorted = df_q4_text_embeddings.sort_values(by="similarity", ascending=False)

df_q4_sorted

,text,vector,similarity
0,Course - Can I still join the course after the...,"[-0.03958748262081998, -0.06904068936129748, 0...",0.851454
1,Course - Can I follow the course after it fini...,"[-0.08249755003702235, -0.08627811833574725, 0...",0.843659
2,Course - When will the course start? The purpo...,"[-0.07798265715506465, -0.054283763168184285, ...",0.840829
4,How can we contribute to the course? Star the ...,"[-0.08384529091747055, -0.05699563384774924, 0...",0.808601
3,Course - What can I do before the course start...,"[-0.0719690950180108, -0.05147854858194681, 0....",0.775516


#### Question 5 : Selecting the embedding model

In [38]:
import pandas as pd

supported_models = (
    pd.DataFrame(TextEmbedding.list_supported_models())
    .sort_values("dim")
    .drop(columns=["sources", "model_file", "additional_files"])
    .reset_index(drop=True)
)
supported_models.head(3)

,model,description,license,size_in_GB,dim,tasks
0,BAAI/bge-small-en,"Text embeddings, Unimodal (text), English, 512...",mit,0.130,384,{}
1,BAAI/bge-small-en-v1.5,"Text embeddings, Unimodal (text), English, 512...",mit,0.067,384,{}
2,snowflake/snowflake-arctic-embed-xs,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.090,384,{}


#### Question 5 : Indexing with qdrant

In [39]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [40]:
EMBEDDING_DIMENSIONALITY = 384

In [41]:
from qdrant_client import QdrantClient, models

# Initialize Qdrant client
client = QdrantClient("http://localhost:6333")

# Collection details
collection_name = "zoomcamp-hw-week-q6"
embedding_dimension = EMBEDDING_DIMENSIONALITY
model_handle = "BAAI/bge-small-en"

# Track whether we created the collection
created_new_collection = False

# Check existing collections
existing_collections = [col.name for col in client.get_collections().collections]

if collection_name not in existing_collections:
    client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=embedding_dimension,
            distance=models.Distance.COSINE
        )
    )
    print(f"✅ Created collection '{collection_name}'")
    created_new_collection = True
else:
    print(f"ℹ️ Collection '{collection_name}' already exists. Skipping creation.")


ℹ️ Collection 'zoomcamp-hw-week-q6' already exists. Skipping creation.


In [ ]:
documents[0]

In [42]:
# Only upsert if the collection was freshly created
if created_new_collection:
    points = []
    id = 0

    for doc in documents:
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['question'] + ' ' + doc['text'], model=model_handle),
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": doc['course']
            }
        )
        points.append(point)
        id += 1

    client.upsert(collection_name=collection_name, points=points)
    print(f"🚀 Upserted {len(points)} documents into '{collection_name}'")

In [43]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword" # exact matching on string metadata fields
    )

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

In [44]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [45]:
search('I just discovered the course. Can I join now?')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])